<a href="https://colab.research.google.com/github/littleadam/AI_ML/blob/main/vector_db/semantic_search_pinecone_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 10.9 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

# 1. Create Sentence Embeddings

# Load pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example sentences
sentences = ["How is the weather today?", "What is the capital of France?", "I need help with my order.","Messi is a good player"]

# Generate embeddings for the sentences
embeddings = model.encode(sentences).tolist()


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(len(embeddings[0]))

384


In [ ]:
!pip install -qU \
  pinecone-client==3.1.0 \
  pandas==2.0.3
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
##api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
api_key = '24df3a07-00c4-4dea-bd4d-cbb487ec1946'

# configure client
pc = Pinecone(api_key=api_key)

from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

# Giving our index a name
index_name = "food-review"

# Delete the index, if an index of the same name already exists
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

import time

dimensions = len(embeddings[0])
pc.create_index(
    name=index_name,
    dimension=dimensions,
    metric="cosine",
    spec=spec
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

In [ ]:

def embed_and_upsert(sentences):
  # Generate embeddings for the sentences
  embeddings = model.encode(sentences).tolist()

  items_to_upsert = [(str(i), embeddings[i]) for i in range(len(embeddings))]
  index.upsert(vectors=items_to_upsert)

def process_query_and_print_results(query):
  # Generate embedding for the query
  query_embedding = model.encode([query]).tolist()

  # Perform search in Pinecone index (top_k=3 returns the top 3 most similar vectors)
  result = index.query(vector=query_embedding[0], top_k=3)
  print("Search Results:")
  for match in result['matches']:
    sentence_id = match['id']  # ID of the matched sentence
    similarity_score = match['score']  # Similarity score
    print(f"ID: {sentence_id}, Similarity Score: {similarity_score}")
    print(f"Matched Sentence: {reviews[int(sentence_id)]}")

In [ ]:
import pandas as pd
# Load the dataset
data = pd.read_csv("/content/Reviews.csv",skiprows=0)

# Sample 2000 reviews randomly
sampled_data = data.sample(1000)
reviews = sampled_data['Text'].tolist()

embed_and_upsert(reviews)
query_ques = "indian food"
print(f"The Question is {query_ques}")
process_query_and_print_results(query_ques)

The Question is fruit food
Search Results:
ID: 509, Similarity Score: 0.569454789
Matched Sentence: Not what I was expecting and will not order again. I was looking for dried fruit. This product was powdery and what wasn't was hard.
ID: 129, Similarity Score: 0.562217355
Matched Sentence: Great standby product for drinking.  BUT everyone who has my fruit salad is curious why it is different and delicious.
ID: 626, Similarity Score: 0.532710671
Matched Sentence: this is a great snack- it is sour and sweet at the same time- I bought these because whenever I bought the Costco mango and berries mix the sour mango always got finished first.  Its also great in salads.
